In [ ]:
import pandas as pd
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch

# Optional: Use GPU if available
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to: {'cuda' if device == 0 else 'cpu'}")

# Load model & tokenizer
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Create pipeline
pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    max_length=64,
    truncation=True,
    padding='max_length',
    device=device,
    return_all_scores=True  # important for score extraction
)

# Label map (depends on pipeline output)
label_map = {
    "LABEL_0": "Bearish",
    "LABEL_1": "Neutral",
    "LABEL_2": "Bullish"
}

# Load your CSV with a 'posts' column
df = pd.read_csv("crypto_posts.csv")  # e.g., ["posts"]

# Process each post to extract top label and score
def predict_sentiment(text):
    scores = pipe(text)[0]  # list of dicts with label/score
    top = max(scores, key=lambda x: x['score'])
    label = label_map.get(top['label'], top['label'])
    return pd.Series([label, top['score']])

# Apply to all rows
df[["sentiment_label", "sentiment_score"]] = df["posts"].apply(predict_sentiment)

# Save result
df.to_csv("crypto_posts_with_sentiment.csv", index=False)
print("✅ Done! File saved as 'crypto_posts_with_sentiment.csv'")
